## Import

In [1]:
!pip install googletrans==3.1.0a0
!pip install sentence_transformers
!pip install soyclustering
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from tqdm import tqdm
from typing import Tuple
import re
import random
import copy
import googletrans
import warnings
warnings.filterwarnings(action='ignore')

from sentence_transformers import SentenceTransformer, util
from scipy.sparse import csr_matrix
from soyclustering import SphericalKMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

from catboost import CatBoostClassifier

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("submission.csv")

In [4]:
final_train = pd.DataFrame()
final_test = pd.DataFrame()

## Translate

In [5]:
translator = googletrans.Translator()

train['product_category'] = train['product_category'].fillna('none')
pc_unique = list(train['product_category'].unique())

def text2eng(text1):
    try:
        return translator.translate(text1, dest='en').text
    except:
        time.sleep(2)
        return translator.translate(text1, dest='en').text
        
pc_dict = dict()

for i in tqdm(pc_unique):
    pc_dict[i] = text2eng(i)

def pc2eng(x):
    return pc_dict[x]

final_train['product_category'] = train['product_category'].apply(pc2eng)

100%|██████████| 358/358 [01:14<00:00,  4.79it/s]


In [6]:
EDA_train = train.copy()
EDA_train['product_category'] = final_train['product_category']
EDA_train.to_csv("EDA_train.csv", index=False)

In [7]:
translator = googletrans.Translator()

test['product_category'] = test['product_category'].fillna('none')
pc_unique_test = list(test['product_category'].unique())

def text2eng(text1):
    try:
        return translator.translate(text1, dest='en').text
    except:
        time.sleep(2)
        return translator.translate(text1, dest='en').text
        
pc_dict_test = dict()

for i in tqdm(pc_unique_test):
    pc_dict_test[i] = text2eng(i)

def pc2eng_test(x):
    return pc_dict_test[x]

final_test['product_category'] = test['product_category'].apply(pc2eng_test)

100%|██████████| 37/37 [00:07<00:00,  4.95it/s]


In [8]:
EDA_test = test.copy()

EDA_test['product_category'] = final_test['product_category']
EDA_test.to_csv("EDA_test.csv", index=False)

## bant_submit

In [9]:
def bant(x):
    if x=='none':
        return 0
    else:
        return 1

In [10]:
final_train[['inquiry_type','customer_position','expected_timeline','expected_budget']] = train[['inquiry_type','customer_position','expected_timeline','expected_budget']].fillna('none')

final_train['inquiry_type'] = train['inquiry_type'].apply(bant)
final_train['customer_position'] = train['customer_position'].apply(bant)
final_train['expected_timeline'] = train['expected_timeline'].apply(bant)
final_train['expected_budget'] = train['expected_budget'].apply(bant)

In [11]:
final_test[['inquiry_type','customer_position','expected_timeline']] = test[['inquiry_type','customer_position','expected_timeline']].fillna('none')

final_test['inquiry_type'] = test['inquiry_type'].apply(bant)
final_test['customer_position'] = test['customer_position'].apply(bant)
final_test['expected_timeline'] = test['expected_timeline'].apply(bant)
final_test['expected_budget'] = test['expected_budget'].apply(bant)

## product/business NaN

In [12]:
def _is_nan_(x):
    if x=='none':
        return 0
    else:
        return 1

In [13]:
final_train['business_area'] = train['business_area'].fillna('none').apply(_is_nan_)
final_train['business_subarea'] = train['business_subarea'].fillna('none').apply(_is_nan_)

final_train['business'] = final_train['business_area'] + final_train['business_subarea']
final_train.drop(['business_area','business_subarea'], axis=1, inplace=True)

In [14]:
final_test['business_area'] = test['business_area'].fillna('none').apply(_is_nan_)
final_test['business_subarea'] = test['business_subarea'].fillna('none').apply(_is_nan_)

final_test['business'] = final_test['business_area'] + final_test['business_subarea']
final_test.drop(['business_area','business_subarea'], axis=1, inplace=True)

In [15]:
final_train['product_category'] = train['product_category'].fillna('none').apply(_is_nan_)
final_train['product_subcategory'] = train['product_subcategory'].fillna('none').apply(_is_nan_)

final_train['product'] = final_train['product_category'] + final_train['product_subcategory']
final_train.drop(['product_category','product_subcategory'], axis=1, inplace=True)

In [16]:
final_test['product_category'] = test['product_category'].fillna('none').apply(_is_nan_)
final_test['product_subcategory'] = test['product_subcategory'].fillna('none').apply(_is_nan_)

final_test['product'] = final_test['product_category'] + final_test['product_subcategory']
final_test.drop(['product_category','product_subcategory'], axis=1, inplace=True)

## product_category

In [17]:
def new_clean(x):
    x = x.lower()
    return x

In [18]:
pattern1 = re.compile('signage')
pattern2 = re.compile('tv')

def pattern_product(x):
    if pattern1.search(x):
        return 'signage/tv'
    elif pattern2.search(x):
        return 'signage/tv'
    else:
        if x!='none':
            return 'nsignage/ntv'
        return x

In [19]:
final_train['product_category'] = EDA_train['product_category'].apply(new_clean)
final_train['product_category'] = EDA_train['product_category'].apply(pattern_product)

In [20]:
final_test['product_category'] = EDA_test['product_category'].apply(new_clean)
final_test['product_category'] = EDA_test['product_category'].apply(pattern_product)

## customer_idx

In [21]:
final_train['customer_idx'] = 0
final_train.loc[(train['customer_idx']==47466),'customer_idx'] = 1
final_train.loc[(train['customer_idx']==25096),'customer_idx'] = 2

In [22]:
final_test['customer_idx'] = 0
final_test.loc[(test['customer_idx']==47466),'customer_idx'] = 1
final_test.loc[(test['customer_idx']==25096),'customer_idx'] = 2

## enterprise

In [23]:
def fe(x):
    return x[0]

In [24]:
re_train = train.copy()

two_enter_idx = train.groupby('customer_idx')['enterprise'].unique()[train.groupby('customer_idx')['enterprise'].unique().apply(len)==2].index
temp_df = train.groupby('customer_idx')['enterprise'].unique()[train.groupby('customer_idx')['enterprise'].unique().apply(len)==2].apply(fe)

for i in tqdm(two_enter_idx):
    re_train.loc[re_train['customer_idx']==i, 'enterprise'] = temp_df[i]

final_train['enterprise'] = re_train['enterprise']

100%|██████████| 572/572 [00:00<00:00, 1804.01it/s]


In [25]:
re2train = re_train[['customer_idx', 'enterprise']].value_counts().reset_index().drop(['count'], axis=1)
re2train = re2train.set_index(re2train['customer_idx']).drop(['customer_idx'], axis=1)
ci_dict = re2train.to_dict()['enterprise']

re_test = test.copy()

for i in tqdm(range(len(re_test))):
    try: 
        re_test.loc[i, 'enterprise'] = ci_dict[re_test.loc[i, 'customer_idx']]
    except:
        continue

final_test['enterprise'] = re_test['enterprise']

100%|██████████| 21341/21341 [00:01<00:00, 19755.77it/s]


## customer_type

In [26]:
def clean(x):
    x = re.sub(r'[^a-zA-Z]', ' ', x)
    x = x.lower()
    return x

In [27]:
pattern = re.compile('end')
def pattern_end(x):
    if pattern.search(x):
        return 'end'
    else:
        if x!='none':
            return 'nend'
        return x

In [28]:
final_train['customer_type'] = train['customer_type'].fillna('none')
final_train['customer_type'] = final_train['customer_type'].apply(clean)
final_train['customer_type'] = final_train['customer_type'].apply(pattern_end)

In [29]:
final_test['customer_type'] = test['customer_type'].fillna('none')
final_test['customer_type'] = final_test['customer_type'].apply(clean)
final_test['customer_type'] = final_test['customer_type'].apply(pattern_end)

## historical_exisiting_cnt

In [30]:
def historical(x):
    if x==-1:
        return 0
    elif x==0:
        return 1
    else:
        return 2

In [31]:
final_train['historical_existing_cnt'] = train['historical_existing_cnt'].fillna(-1).apply(historical)

In [32]:
final_test['historical_existing_cnt'] = test['historical_existing_cnt'].fillna(-1).apply(historical)

## others
- response_corporate, business_unit, id_strategic_ver, it_strategic_ver, lead_owner, ver_cus, ver_pro

In [33]:
final_train['response_corporate'] = train['response_corporate']
final_train['business_unit'] = train['business_unit']
final_train['id_strategic_ver'] = train['id_strategic_ver'].fillna(0)
final_train['it_strategic_ver'] = train['it_strategic_ver'].fillna(0)
final_train['lead_owner'] = train['lead_owner']
final_train[['ver_cus','ver_pro']] = train[['ver_cus','ver_pro']]

In [34]:
final_test['response_corporate'] = test['response_corporate']
final_test['business_unit'] = test['business_unit']
final_test['id_strategic_ver'] = test['id_strategic_ver'].fillna(0)
final_test['it_strategic_ver'] = test['it_strategic_ver'].fillna(0)
final_test['lead_owner'] = test['lead_owner']
final_test[['ver_cus','ver_pro']] = test[['ver_cus','ver_pro']]

## prefer_ver_count

In [35]:
train_prefer = train.copy()
test_prefer = test.copy()

In [36]:
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='ID'][train_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 0

train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='AS'][train_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 2

train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='IT'][train_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 1

train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
train_prefer.loc[(train_prefer.index.isin(list(train_prefer[train_prefer['business_unit']=='Solution'][train_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 0

train_prefer['prefer_ver_count'].fillna(0, inplace=True)

In [37]:
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='ID'][test_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 0

test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='AS'][test_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 2

test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 2
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='IT'][test_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 1

test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='hospital & health care']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='education']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='retail']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='corporate / office']['prefer_ver_count'].index))),'prefer_ver_count'] = 3
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='government department']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='residential (home)']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='public facility']['prefer_ver_count'].index))),'prefer_ver_count'] = 0
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='hotel & accommodation']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='special purpose']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='factory']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='transportation']['prefer_ver_count'].index))),'prefer_ver_count'] = 1
test_prefer.loc[(test_prefer.index.isin(list(test_prefer[test_prefer['business_unit']=='Solution'][test_prefer['business_area']=='power plant / renewable energy']['prefer_ver_count'].index))),'prefer_ver_count'] = 0

test_prefer['prefer_ver_count'].fillna(0, inplace=True)

In [38]:
final_train['prefer_ver_count'] = train_prefer['prefer_ver_count']
final_test['prefer_ver_count'] = test_prefer['prefer_ver_count']

## lead_from_channel

In [39]:
pattern1 = re.compile('facebook')
pattern2 = re.compile('webinar')

def pattern_channel(x):
    if pattern1.search(x):
        return 'facebook/webinar'
    elif pattern2.search(x):
        return 'facebook/webinar'
    else:
        if x!='none':
            return 'others'
        return x

In [40]:
final_train['lead_from_channel'] = train['lead_from_channel'].fillna('none').apply(pattern_channel)

In [41]:
final_test['lead_from_channel'] = test['lead_from_channel'].fillna('none').apply(pattern_channel)

## is_converted

In [42]:
final_train['is_converted'] = train['is_converted']

## datetime

In [43]:
final_train['lead_date'] = train['lead_date']
final_train['lead_date'] = pd.to_datetime(final_train['lead_date'])

final_train = final_train.sort_values(['lead_date']).reset_index(drop=True)
final_train

,inquiry_type,customer_position,expected_timeline,expected_budget,business,product,product_category,customer_idx,enterprise,customer_type,...,business_unit,id_strategic_ver,it_strategic_ver,lead_owner,ver_cus,ver_pro,prefer_ver_count,lead_from_channel,is_converted,lead_date
0,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,none,...,ID,0.0,0.0,262,0,0,0.0,others,False,2021-01-01
1,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,none,...,ID,0.0,0.0,179,0,0,0.0,others,False,2021-01-01
2,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,none,...,ID,0.0,0.0,179,0,0,0.0,others,False,2021-01-01
3,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,none,...,ID,0.0,0.0,262,0,0,0.0,others,False,2021-01-01
4,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,none,...,ID,0.0,0.0,262,0,0,0.0,others,False,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1,1,1,1,0,1,nsignage/ntv,0,SMB,nend,...,AS,0.0,0.0,694,0,0,0.0,others,False,2023-05-31
59295,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,end,...,AS,0.0,0.0,97,0,0,0.0,others,False,2023-05-31
59296,1,1,1,1,1,1,nsignage/ntv,0,SMB,nend,...,AS,0.0,0.0,638,0,0,2.0,others,False,2023-05-31
59297,1,0,1,1,0,1,nsignage/ntv,1,Enterprise,end,...,AS,0.0,0.0,115,0,0,0.0,others,False,2023-05-31


In [44]:
split_date = final_train.iloc[-len(final_train)//10]['lead_date']

MOE_train_date = list(set(final_train[final_train['lead_date']<split_date]['lead_date']))
MOE_val_date = list(set(final_train[final_train['lead_date']>=split_date]['lead_date']))

In [45]:
MOE_train = final_train[final_train['lead_date'].isin(MOE_train_date)]
MOE_val = final_train[final_train['lead_date'].isin(MOE_val_date)]

## Experts

In [46]:
YJ_columns = ['business','response_corporate','lead_owner','business_unit','product_category','customer_type','historical_existing_cnt','lead_from_channel','customer_idx','is_converted']
YL_columns = ['inquiry_type','customer_position','expected_timeline','expected_budget','business','product','product_category','customer_idx','enterprise','customer_type','historical_existing_cnt',
              'response_corporate','business_unit','id_strategic_ver','it_strategic_ver','lead_owner','ver_cus','ver_pro','prefer_ver_count','is_converted']

YJ_columns_test = ['business','response_corporate','lead_owner','business_unit','product_category','customer_type','historical_existing_cnt','lead_from_channel','customer_idx']
YL_columns_test = ['inquiry_type','customer_position','expected_timeline','expected_budget','business','product','product_category','customer_idx','enterprise','customer_type','historical_existing_cnt',
              'response_corporate','business_unit','id_strategic_ver','it_strategic_ver','lead_owner','ver_cus','ver_pro','prefer_ver_count']

In [47]:
YJ_train = MOE_train[YJ_columns]
YL_train = MOE_train[YL_columns]

In [48]:
YJ_val = MOE_val[YJ_columns]
YL_val = MOE_val[YL_columns]

In [49]:
YJ_test = final_test[YJ_columns_test]
YL_test = final_test[YL_columns_test]

In [70]:
y_tra = MOE_train['is_converted']
y_val = MOE_val['is_converted']

## YL train

In [51]:
def toint(x):
    return int(x)

In [52]:
YL_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53042 entries, 0 to 53041
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   inquiry_type             53042 non-null  int64  
 1   customer_position        53042 non-null  int64  
 2   expected_timeline        53042 non-null  int64  
 3   expected_budget          53042 non-null  int64  
 4   business                 53042 non-null  int64  
 5   product                  53042 non-null  int64  
 6   product_category         53042 non-null  object 
 7   customer_idx             53042 non-null  int64  
 8   enterprise               53042 non-null  object 
 9   customer_type            53042 non-null  object 
 10  historical_existing_cnt  53042 non-null  int64  
 11  response_corporate       53042 non-null  object 
 12  business_unit            53042 non-null  object 
 13  id_strategic_ver         53042 non-null  float64
 14  it_strategic_ver         53

In [53]:
train_fin = YL_train.copy()
train_fin.columns = [f"{i}" for i in range(len(train_fin.columns)-1)]+['is_converted']

X_fin_test = YL_val.copy()
X_fin_test.columns = [f"{i}" for i in range(len(X_fin_test.columns))]

MOE_test = YL_test.copy()
MOE_test.columns = [f"{i}" for i in range(len(MOE_test.columns))]

numeric_category = [0,1,2,3,4,5,7,10,13,14,15,16,17,18]
total_category = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

for i in numeric_category:
    exec(f"train_fin['{i}'] = train_fin['{i}'].apply(toint)")
    exec(f"X_fin_test['{i}'] = X_fin_test['{i}'].apply(toint)")
    exec(f"MOE_test['{i}'] = MOE_test['{i}'].apply(toint)")

In [54]:
train_down_5split = train_fin

df_label_0 = train_down_5split[train_down_5split['is_converted'] == 0]
df_label_1 = train_down_5split[train_down_5split['is_converted'] == 1]

df_label_0_shuffled = df_label_0.sample(frac=1, random_state=42) 
# df_label_0_shuffled = df_label_0_shuffled.iloc[:1*10*(len(df_label_1)),:]
df_label_0_shuffled = df_label_0_shuffled.iloc[:,:]

num_parts = 2
df_label_0_splits = np.array_split(df_label_0_shuffled, num_parts)

df_label_0_split_dfs = []
for i, split in enumerate(df_label_0_splits, start=1):
    df_label_0_split_dfs.append(pd.DataFrame(split))

for i, df_split in enumerate(df_label_0_split_dfs, start=1):
    print(f"Length of split {i}: {len(df_split)}")

Length of split 1: 24584
Length of split 2: 24583


In [55]:
total_result1 = pd.DataFrame([])
total_result2 = pd.DataFrame([])
total_result3 = pd.DataFrame([])
val_f1 = []
train_f1 = []
list_t = []
n=0

for i in range(num_parts):
    print(i)
    exec(f"train_{i} = pd.concat([df_label_0_split_dfs[{i}],df_label_1]).reset_index(drop=True)")
    exec(f"train_{i} = train_{i}.sample(len(train_{i}), random_state=42).reset_index(drop=True)")
    X = eval(f"train_{i}").drop(columns=['is_converted'])
    y = eval(f"train_{i}")['is_converted'].astype(int)

    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 5)
#     k_fold_on = 1
    
    for train_index, test_index in str_kf.split(X,y):
#         if k_fold_on == 0:
#             break
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        class CatBoostEvalMetricPearson(object): 
            global global_t
            def __init__(self): 
                global global_t
                self.best_t = 0
                self.best_f1 = 0
                global_t = self.best_t

            def get_final_error(self, error, weight):
                return error

            def is_max_optimal(self):
                return True

            def evaluate(self, approxes, target, weight):
                global global_t
                assert len(approxes) == 1
                assert len(target) == len(approxes[0])
                logit_train = 1/(1+np.exp(-approxes[0]))
                target_train = np.array(target)
                
                best_f1 = 0
                best_t = 0
                for temp_idx in range(50,51,1): # (20, 91, 5) (50,51,1)
                    temp_t = temp_idx/100
                    preds_train = np.zeros(approxes[0].shape)
                    preds_train[logit_train >= temp_t] = 1
                    preds_train[logit_train < temp_t] = 0
                    temp_f1 = f1_score(target_train, preds_train)
                    if (best_f1 < temp_f1):
                        best_f1 = temp_f1
                        best_t = temp_t
                        if (self.best_f1 < best_f1) and (len(target)==len(X_test)):
                            self.best_f1 = best_f1
                            self.best_t = best_t
                            global_t = best_t
                            print(self.best_f1, self.best_t)
                return best_f1, 0
            

        X_train = np.array(X_train).tolist()
        X_test = np.array(X_test).tolist()
        y_train = np.array(y_train).tolist()
        y_test = np.array(y_test).tolist()

        model_cat = CatBoostClassifier(learning_rate = 0.1, 
                                    depth = 8, 
                                    iterations = 3000, 
                                    random_state = 100, 
                                    objective='Logloss',
                                    eval_metric=CatBoostEvalMetricPearson(),
                                    scale_pos_weight=1,###
                                    one_hot_max_size=1)

        model_cat.fit(X_train, y_train, 
                    eval_set=[(X_test, y_test)],
                    cat_features=total_category,
                    early_stopping_rounds=100,### 
                    use_best_model=True)
        
        print(global_t)
        list_t.append(global_t)

        model_cat.set_probability_threshold(global_t)
        temp_result1 = model_cat.predict(train_fin)
        temp_result2 = model_cat.predict(X_fin_test)
        temp_result3 = model_cat.predict(MOE_test)

        temp_result_np1 = np.array(temp_result1).reshape(-1,1)
        temp_result_np2 = np.array(temp_result2).reshape(-1,1)
        temp_result_np3 = np.array(temp_result3).reshape(-1,1)
        total_result1 = pd.concat([total_result1,pd.DataFrame(temp_result_np1)],axis=1)
        total_result2 = pd.concat([total_result2,pd.DataFrame(temp_result_np2)],axis=1)
        total_result3 = pd.concat([total_result3,pd.DataFrame(temp_result_np3)],axis=1)

        val_f1.append(f1_score(pd.DataFrame(y_test),model_cat.predict(pd.DataFrame(X_test))))
        train_f1.append(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))
        print(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))

        exec(f"model_cat.save_model('model_cat_{n}')")
        n+=1
#         k_fold_on = 0
    

0
0.7824037706205813 0.5
0:	learn: 0.7653731	test: 0.7824038	best: 0.7824038 (0)	total: 141ms	remaining: 7m 1s
1:	learn: 0.7718744	test: 0.7824038	best: 0.7824038 (0)	total: 228ms	remaining: 5m 41s
2:	learn: 0.7660167	test: 0.7824038	best: 0.7824038 (0)	total: 275ms	remaining: 4m 34s
3:	learn: 0.7655255	test: 0.7824038	best: 0.7824038 (0)	total: 312ms	remaining: 3m 53s
4:	learn: 0.7659236	test: 0.7824038	best: 0.7824038 (0)	total: 384ms	remaining: 3m 49s
0.7866043613707165 0.5
5:	learn: 0.7736333	test: 0.7866044	best: 0.7866044 (5)	total: 437ms	remaining: 3m 37s
6:	learn: 0.7730860	test: 0.7856586	best: 0.7866044 (5)	total: 479ms	remaining: 3m 24s
7:	learn: 0.7746257	test: 0.7866044	best: 0.7866044 (5)	total: 536ms	remaining: 3m 20s
8:	learn: 0.7768822	test: 0.7866044	best: 0.7866044 (5)	total: 594ms	remaining: 3m 17s
0.7935034802784223 0.5
9:	learn: 0.7830336	test: 0.7935035	best: 0.7935035 (9)	total: 650ms	remaining: 3m 14s
0.8018433179723502 0.5
10:	learn: 0.7881026	test: 0.8018433	

0.895263509006004 0.5
88:	learn: 0.8968686	test: 0.8952635	best: 0.8952635 (88)	total: 5.58s	remaining: 3m 2s
89:	learn: 0.8973966	test: 0.8952635	best: 0.8952635 (88)	total: 5.65s	remaining: 3m 2s
90:	learn: 0.8972090	test: 0.8952635	best: 0.8952635 (88)	total: 5.72s	remaining: 3m 2s
0.8967355096602265 0.5
91:	learn: 0.8982993	test: 0.8967355	best: 0.8967355 (91)	total: 5.79s	remaining: 3m 2s
92:	learn: 0.8984521	test: 0.8967355	best: 0.8967355 (91)	total: 5.85s	remaining: 3m 2s
0.8973333333333334 0.5
93:	learn: 0.8984867	test: 0.8973333	best: 0.8973333 (93)	total: 5.91s	remaining: 3m 2s
0.8986666666666666 0.5
94:	learn: 0.8982993	test: 0.8986667	best: 0.8986667 (94)	total: 5.97s	remaining: 3m 2s
95:	learn: 0.8982993	test: 0.8986667	best: 0.8986667 (94)	total: 6.04s	remaining: 3m 2s
96:	learn: 0.8989108	test: 0.8983957	best: 0.8986667 (94)	total: 6.1s	remaining: 3m 2s
0.9004676018704074 0.5
97:	learn: 0.9006803	test: 0.9004676	best: 0.9004676 (97)	total: 6.16s	remaining: 3m 2s
98:	lea

178:	learn: 0.9271657	test: 0.8977956	best: 0.9024064 (128)	total: 11.3s	remaining: 2m 58s
179:	learn: 0.9270098	test: 0.8977956	best: 0.9024064 (128)	total: 11.4s	remaining: 2m 58s
180:	learn: 0.9271902	test: 0.8970588	best: 0.9024064 (128)	total: 11.4s	remaining: 2m 57s
181:	learn: 0.9271902	test: 0.8976589	best: 0.9024064 (128)	total: 11.5s	remaining: 2m 57s
182:	learn: 0.9275508	test: 0.8976589	best: 0.9024064 (128)	total: 11.6s	remaining: 2m 57s
183:	learn: 0.9278870	test: 0.8976589	best: 0.9024064 (128)	total: 11.6s	remaining: 2m 57s
184:	learn: 0.9280430	test: 0.8983957	best: 0.9024064 (128)	total: 11.7s	remaining: 2m 57s
185:	learn: 0.9285594	test: 0.8983957	best: 0.9024064 (128)	total: 11.7s	remaining: 2m 57s
186:	learn: 0.9288956	test: 0.8965977	best: 0.9024064 (128)	total: 11.8s	remaining: 2m 57s
187:	learn: 0.9288956	test: 0.8971963	best: 0.9024064 (128)	total: 11.9s	remaining: 2m 57s
188:	learn: 0.9292080	test: 0.8971963	best: 0.9024064 (128)	total: 11.9s	remaining: 2m 57s

36:	learn: 0.8805532	test: 0.8628809	best: 0.8630994 (28)	total: 2.3s	remaining: 3m 4s
37:	learn: 0.8807181	test: 0.8628809	best: 0.8630994 (28)	total: 2.37s	remaining: 3m 4s
38:	learn: 0.8811044	test: 0.8628809	best: 0.8630994 (28)	total: 2.43s	remaining: 3m 4s
0.8636678200692043 0.5
39:	learn: 0.8813384	test: 0.8636678	best: 0.8636678 (39)	total: 2.5s	remaining: 3m 4s
40:	learn: 0.8812274	test: 0.8636678	best: 0.8636678 (39)	total: 2.56s	remaining: 3m 4s
41:	learn: 0.8816945	test: 0.8630705	best: 0.8636678 (39)	total: 2.63s	remaining: 3m 4s
42:	learn: 0.8825047	test: 0.8632597	best: 0.8636678 (39)	total: 2.68s	remaining: 3m 4s
43:	learn: 0.8823529	test: 0.8632597	best: 0.8636678 (39)	total: 2.71s	remaining: 3m 1s
0.8638562543192811 0.5
44:	learn: 0.8834651	test: 0.8638563	best: 0.8638563 (44)	total: 2.78s	remaining: 3m 2s
45:	learn: 0.8838089	test: 0.8632597	best: 0.8638563 (44)	total: 2.84s	remaining: 3m 2s
0.8640441683919944 0.5
46:	learn: 0.8849254	test: 0.8640442	best: 0.8640442 

126:	learn: 0.9151279	test: 0.8698023	best: 0.8736413 (92)	total: 7.87s	remaining: 2m 58s
127:	learn: 0.9144793	test: 0.8698023	best: 0.8736413 (92)	total: 7.93s	remaining: 2m 57s
128:	learn: 0.9155525	test: 0.8692098	best: 0.8736413 (92)	total: 7.99s	remaining: 2m 57s
129:	learn: 0.9154263	test: 0.8693878	best: 0.8736413 (92)	total: 8.06s	remaining: 2m 57s
130:	learn: 0.9155811	test: 0.8687967	best: 0.8736413 (92)	total: 8.12s	remaining: 2m 57s
131:	learn: 0.9162861	test: 0.8687967	best: 0.8736413 (92)	total: 8.19s	remaining: 2m 57s
132:	learn: 0.9164694	test: 0.8699796	best: 0.8736413 (92)	total: 8.25s	remaining: 2m 57s
133:	learn: 0.9168076	test: 0.8699796	best: 0.8736413 (92)	total: 8.3s	remaining: 2m 57s
134:	learn: 0.9168076	test: 0.8692098	best: 0.8736413 (92)	total: 8.37s	remaining: 2m 57s
135:	learn: 0.9169907	test: 0.8707483	best: 0.8736413 (92)	total: 8.44s	remaining: 2m 57s
136:	learn: 0.9164694	test: 0.8713410	best: 0.8736413 (92)	total: 8.5s	remaining: 2m 57s
137:	learn: 

...

0.8688873139617292 0.5
15:	learn: 0.8343401	test: 0.8688873	best: 0.8688873 (15)	total: 990ms	remaining: 3m 4s
16:	learn: 0.8340938	test: 0.8676056	best: 0.8688873 (15)	total: 1.05s	remaining: 3m 4s
0.8731604765241766 0.5
17:	learn: 0.8384755	test: 0.8731605	best: 0.8731605 (17)	total: 1.11s	remaining: 3m 3s
18:	learn: 0.8399349	test: 0.8729050	best: 0.8731605 (17)	total: 1.18s	remaining: 3m 5s
19:	learn: 0.8456835	test: 0.8728284	best: 0.8731605 (17)	total: 1.24s	remaining: 3m 4s
20:	learn: 0.8495797	test: 0.8723994	best: 0.8731605 (17)	total: 1.3s	remaining: 3m 4s
21:	learn: 0.8499644	test: 0.8713693	best: 0.8731605 (17)	total: 1.35s	remaining: 3m 3s
22:	learn: 0.8528838	test: 0.8725017	best: 0.8731605 (17)	total: 1.42s	remaining: 3m 3s
0.8743994509265615 0.5
23:	learn: 0.8559831	test: 0.8743995	best: 0.8743995 (23)	total: 1.48s	remaining: 3m 2s
0.8776486671223513 0.5
24:	learn: 0.8577457	test: 0.8776487	best: 0.8776487 (24)	total: 1.54s	remaining: 3m 3s
25:	learn: 0.8609016	test: 0.

0.891102257636122 0.5
108:	learn: 0.9032367	test: 0.8911023	best: 0.8911023 (108)	total: 6.68s	remaining: 2m 57s
109:	learn: 0.9046248	test: 0.8911023	best: 0.8911023 (108)	total: 6.74s	remaining: 2m 57s
0.8918380889183809 0.5
110:	learn: 0.9041328	test: 0.8918381	best: 0.8918381 (110)	total: 6.8s	remaining: 2m 57s
111:	learn: 0.9042860	test: 0.8918381	best: 0.8918381 (110)	total: 6.86s	remaining: 2m 56s
112:	learn: 0.9048103	test: 0.8902196	best: 0.8918381 (110)	total: 6.92s	remaining: 2m 56s
113:	learn: 0.9047780	test: 0.8915502	best: 0.8918381 (110)	total: 6.99s	remaining: 2m 56s
114:	learn: 0.9051491	test: 0.8911023	best: 0.8918381 (110)	total: 7.06s	remaining: 2m 57s
115:	learn: 0.9053345	test: 0.8911023	best: 0.8918381 (110)	total: 7.12s	remaining: 2m 57s
116:	learn: 0.9053345	test: 0.8918381	best: 0.8918381 (110)	total: 7.18s	remaining: 2m 56s
0.8922872340425532 0.5
117:	learn: 0.9055518	test: 0.8922872	best: 0.8922872 (117)	total: 7.25s	remaining: 2m 56s
118:	learn: 0.9055838	t

199:	learn: 0.9287999	test: 0.8915187	best: 0.8940397 (123)	total: 12.4s	remaining: 2m 54s
200:	learn: 0.9289801	test: 0.8907895	best: 0.8940397 (123)	total: 12.5s	remaining: 2m 54s
201:	learn: 0.9289801	test: 0.8913759	best: 0.8940397 (123)	total: 12.6s	remaining: 2m 53s
202:	learn: 0.9295206	test: 0.8905013	best: 0.8940397 (123)	total: 12.6s	remaining: 2m 53s
203:	learn: 0.9297006	test: 0.8912327	best: 0.8940397 (123)	total: 12.7s	remaining: 2m 53s
204:	learn: 0.9298806	test: 0.8906456	best: 0.8940397 (123)	total: 12.8s	remaining: 2m 53s
205:	learn: 0.9298806	test: 0.8906456	best: 0.8940397 (123)	total: 12.8s	remaining: 2m 53s
206:	learn: 0.9308430	test: 0.8905013	best: 0.8940397 (123)	total: 12.9s	remaining: 2m 53s
207:	learn: 0.9308430	test: 0.8899143	best: 0.8940397 (123)	total: 12.9s	remaining: 2m 53s
208:	learn: 0.9303499	test: 0.8899143	best: 0.8940397 (123)	total: 13s	remaining: 2m 53s
209:	learn: 0.9313131	test: 0.8899143	best: 0.8940397 (123)	total: 13.1s	remaining: 2m 53s
2

62:	learn: 0.8851675	test: 0.8616438	best: 0.8637919 (56)	total: 3.69s	remaining: 2m 51s
0.8645690834473324 0.5
63:	learn: 0.8849044	test: 0.8645691	best: 0.8645691 (63)	total: 3.75s	remaining: 2m 51s
0.865160848733744 0.5
64:	learn: 0.8856265	test: 0.8651608	best: 0.8651608 (64)	total: 3.81s	remaining: 2m 52s
65:	learn: 0.8862357	test: 0.8651608	best: 0.8651608 (64)	total: 3.88s	remaining: 2m 52s
0.86593707250342 0.5
66:	learn: 0.8864644	test: 0.8659371	best: 0.8659371 (66)	total: 3.95s	remaining: 2m 52s
0.8678986995208762 0.5
67:	learn: 0.8887753	test: 0.8678987	best: 0.8678987 (67)	total: 4.01s	remaining: 2m 52s
68:	learn: 0.8885860	test: 0.8678987	best: 0.8678987 (67)	total: 4.04s	remaining: 2m 51s
69:	learn: 0.8891538	test: 0.8667122	best: 0.8678987 (67)	total: 4.08s	remaining: 2m 50s
70:	learn: 0.8887753	test: 0.8661202	best: 0.8678987 (67)	total: 4.15s	remaining: 2m 51s
71:	learn: 0.8896834	test: 0.8661202	best: 0.8678987 (67)	total: 4.19s	remaining: 2m 50s
0.868259385665529 0.5

154:	learn: 0.9193113	test: 0.8734859	best: 0.8789723 (90)	total: 9.36s	remaining: 2m 51s
155:	learn: 0.9200675	test: 0.8727273	best: 0.8789723 (90)	total: 9.42s	remaining: 2m 51s
156:	learn: 0.9207420	test: 0.8734859	best: 0.8789723 (90)	total: 9.49s	remaining: 2m 51s
157:	learn: 0.9214960	test: 0.8740741	best: 0.8789723 (90)	total: 9.55s	remaining: 2m 51s
158:	learn: 0.9212877	test: 0.8740741	best: 0.8789723 (90)	total: 9.6s	remaining: 2m 51s
159:	learn: 0.9221960	test: 0.8742434	best: 0.8789723 (90)	total: 9.66s	remaining: 2m 51s
160:	learn: 0.9221960	test: 0.8742434	best: 0.8789723 (90)	total: 9.72s	remaining: 2m 51s
161:	learn: 0.9221960	test: 0.8742434	best: 0.8789723 (90)	total: 9.78s	remaining: 2m 51s
162:	learn: 0.9228437	test: 0.8742434	best: 0.8789723 (90)	total: 9.85s	remaining: 2m 51s
163:	learn: 0.9224806	test: 0.8742434	best: 0.8789723 (90)	total: 9.91s	remaining: 2m 51s
164:	learn: 0.9228177	test: 0.8755884	best: 0.8789723 (90)	total: 9.98s	remaining: 2m 51s
165:	learn:

In [56]:
print(np.array(train_f1).mean(), np.array(val_f1).mean(), np.array(val_f1).std())

0.9101406674627939 0.879902443884118 0.010905999269228221


## YJ train

In [57]:
YJ_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53042 entries, 0 to 53041
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   business                 53042 non-null  int64 
 1   response_corporate       53042 non-null  object
 2   lead_owner               53042 non-null  int64 
 3   business_unit            53042 non-null  object
 4   product_category         53042 non-null  object
 5   customer_type            53042 non-null  object
 6   historical_existing_cnt  53042 non-null  int64 
 7   lead_from_channel        53042 non-null  object
 8   customer_idx             53042 non-null  int64 
 9   is_converted             53042 non-null  bool  
dtypes: bool(1), int64(4), object(5)
memory usage: 4.1+ MB


In [58]:
train_fin = YJ_train.copy()
train_fin.columns = [f"{i}" for i in range(len(train_fin.columns)-1)]+['is_converted']

X_fin_test = YJ_val.copy()
X_fin_test.columns = [f"{i}" for i in range(len(X_fin_test.columns))]

MOE_test = YJ_test.copy()
MOE_test.columns = [f"{i}" for i in range(len(MOE_test.columns))]

numeric_category = [0,2,6,8]
total_category = [0,1,2,3,4,5,6,7,8]

for i in numeric_category:
    exec(f"train_fin['{i}'] = train_fin['{i}'].apply(toint)")
    exec(f"X_fin_test['{i}'] = X_fin_test['{i}'].apply(toint)")
    exec(f"MOE_test['{i}'] = MOE_test['{i}'].apply(toint)")

In [59]:
train_down_5split = train_fin

df_label_0 = train_down_5split[train_down_5split['is_converted'] == 0]
df_label_1 = train_down_5split[train_down_5split['is_converted'] == 1]

df_label_0_shuffled = df_label_0.sample(frac=1, random_state=42) 
# df_label_0_shuffled = df_label_0_shuffled.iloc[:1*10*(len(df_label_1)),:]
df_label_0_shuffled = df_label_0_shuffled.iloc[:,:]

num_parts = 2
df_label_0_splits = np.array_split(df_label_0_shuffled, num_parts)

df_label_0_split_dfs = []
for i, split in enumerate(df_label_0_splits, start=1):
    df_label_0_split_dfs.append(pd.DataFrame(split))

for i, df_split in enumerate(df_label_0_split_dfs, start=1):
    print(f"Length of split {i}: {len(df_split)}")

Length of split 1: 24584
Length of split 2: 24583


In [60]:
total_result1_YJ = pd.DataFrame([])
total_result2_YJ = pd.DataFrame([])
total_result3_YJ = pd.DataFrame([])
val_f1 = []
train_f1 = []
list_t = []
n=0

for i in range(num_parts):
    print(i)
    exec(f"train_{i} = pd.concat([df_label_0_split_dfs[{i}],df_label_1]).reset_index(drop=True)")
    exec(f"train_{i} = train_{i}.sample(len(train_{i}), random_state=42).reset_index(drop=True)")
    X = eval(f"train_{i}").drop(columns=['is_converted'])
    y = eval(f"train_{i}")['is_converted'].astype(int)

    str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 5)
#     k_fold_on = 1
    
    for train_index, test_index in str_kf.split(X,y):
#         if k_fold_on == 0:
#             break
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        class CatBoostEvalMetricPearson(object): 
            global global_t
            def __init__(self): 
                global global_t
                self.best_t = 0
                self.best_f1 = 0
                global_t = self.best_t

            def get_final_error(self, error, weight):
                return error

            def is_max_optimal(self):
                return True

            def evaluate(self, approxes, target, weight):
                global global_t
                assert len(approxes) == 1
                assert len(target) == len(approxes[0])
                logit_train = 1/(1+np.exp(-approxes[0]))
                target_train = np.array(target)
                
                best_f1 = 0
                best_t = 0
                for temp_idx in range(50,51,1): # (20, 91, 5) (50,51,1)
                    temp_t = temp_idx/100
                    preds_train = np.zeros(approxes[0].shape)
                    preds_train[logit_train >= temp_t] = 1
                    preds_train[logit_train < temp_t] = 0
                    temp_f1 = f1_score(target_train, preds_train)
                    if (best_f1 < temp_f1):
                        best_f1 = temp_f1
                        best_t = temp_t
                        if (self.best_f1 < best_f1) and (len(target)==len(X_test)):
                            self.best_f1 = best_f1
                            self.best_t = best_t
                            global_t = best_t
                            print(self.best_f1, self.best_t)
                return best_f1, 0
            

        X_train = np.array(X_train).tolist()
        X_test = np.array(X_test).tolist()
        y_train = np.array(y_train).tolist()
        y_test = np.array(y_test).tolist()

        model_cat = CatBoostClassifier(learning_rate = 0.1, 
                                    depth = 5, 
                                    iterations = 3000, 
                                    random_state = 349, 
                                    objective='Logloss',
                                    eval_metric=CatBoostEvalMetricPearson(),
                                    scale_pos_weight=1,
                                    one_hot_max_size=1)

        model_cat.fit(X_train, y_train, 
                    eval_set=[(X_test, y_test)],
                    cat_features=total_category,
                    early_stopping_rounds=50, 
                    use_best_model=True)
        
        print(global_t)
        list_t.append(global_t)

        model_cat.set_probability_threshold(global_t)
        temp_result1_YJ = model_cat.predict(train_fin)
        temp_result2_YJ = model_cat.predict(X_fin_test)
        temp_result3_YJ = model_cat.predict(MOE_test)

        temp_result_np1_YJ = np.array(temp_result1_YJ).reshape(-1,1)
        temp_result_np2_YJ = np.array(temp_result2_YJ).reshape(-1,1)
        temp_result_np3_YJ = np.array(temp_result3_YJ).reshape(-1,1)
        total_result1_YJ = pd.concat([total_result1_YJ,pd.DataFrame(temp_result_np1_YJ)],axis=1)
        total_result2_YJ = pd.concat([total_result2_YJ,pd.DataFrame(temp_result_np2_YJ)],axis=1)
        total_result3_YJ = pd.concat([total_result3_YJ,pd.DataFrame(temp_result_np3_YJ)],axis=1)

        val_f1.append(f1_score(pd.DataFrame(y_test),model_cat.predict(pd.DataFrame(X_test))))
        train_f1.append(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))
        print(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))

        exec(f"model_cat.save_model('model_cat_{n}')")
        n+=1
#         k_fold_on = 0
    

0
0.7824037706205813 0.5
0:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 48ms	remaining: 2m 24s
1:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 83.6ms	remaining: 2m 5s
2:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 121ms	remaining: 2m 1s
3:	learn: 0.7654321	test: 0.7824038	best: 0.7824038 (0)	total: 157ms	remaining: 1m 57s
4:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 193ms	remaining: 1m 55s
5:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 229ms	remaining: 1m 54s
6:	learn: 0.7656779	test: 0.7824038	best: 0.7824038 (0)	total: 269ms	remaining: 1m 55s
0.7862176977290525 0.5
7:	learn: 0.7718187	test: 0.7862177	best: 0.7862177 (7)	total: 308ms	remaining: 1m 55s
8:	learn: 0.7732754	test: 0.7856025	best: 0.7862177 (7)	total: 345ms	remaining: 1m 54s
0.7875486381322958 0.5
9:	learn: 0.7770350	test: 0.7875486	best: 0.7875486 (9)	total: 384ms	remaining: 1m 54s
0.8033794162826421 0.5
10:	learn: 0.7924310	test: 0.8033794	b

89:	learn: 0.8682012	test: 0.8916944	best: 0.8940706 (87)	total: 3.55s	remaining: 1m 54s
90:	learn: 0.8682012	test: 0.8911023	best: 0.8940706 (87)	total: 3.59s	remaining: 1m 54s
0.8948069241011984 0.5
91:	learn: 0.8685422	test: 0.8948069	best: 0.8948069 (91)	total: 3.63s	remaining: 1m 54s
92:	learn: 0.8685422	test: 0.8948069	best: 0.8948069 (91)	total: 3.65s	remaining: 1m 54s
0.8955422488356621 0.5
93:	learn: 0.8681562	test: 0.8955422	best: 0.8955422 (93)	total: 3.69s	remaining: 1m 54s
94:	learn: 0.8682594	test: 0.8955422	best: 0.8955422 (93)	total: 3.73s	remaining: 1m 54s
0.8961384820239681 0.5
95:	learn: 0.8689867	test: 0.8961385	best: 0.8961385 (95)	total: 3.78s	remaining: 1m 54s
96:	learn: 0.8689867	test: 0.8961385	best: 0.8961385 (95)	total: 3.82s	remaining: 1m 54s
0.8976063829787234 0.5
97:	learn: 0.8689420	test: 0.8976064	best: 0.8976064 (97)	total: 3.86s	remaining: 1m 54s
98:	learn: 0.8693279	test: 0.8964143	best: 0.8976064 (97)	total: 3.9s	remaining: 1m 54s
99:	learn: 0.869327

0.7767295597484277 0.5
8:	learn: 0.7913195	test: 0.7767296	best: 0.7767296 (8)	total: 361ms	remaining: 1m 59s
0.7796078431372548 0.5
9:	learn: 0.7935895	test: 0.7796078	best: 0.7796078 (9)	total: 401ms	remaining: 2m
0.7959969207082371 0.5
10:	learn: 0.8079118	test: 0.7959969	best: 0.7959969 (10)	total: 438ms	remaining: 1m 59s
0.8129226145755071 0.5
11:	learn: 0.8156760	test: 0.8129226	best: 0.8129226 (11)	total: 476ms	remaining: 1m 58s
0.8255382331106161 0.5
12:	learn: 0.8215551	test: 0.8255382	best: 0.8255382 (12)	total: 513ms	remaining: 1m 57s
13:	learn: 0.8270510	test: 0.8253500	best: 0.8255382 (12)	total: 549ms	remaining: 1m 57s
0.8281938325991189 0.5
14:	learn: 0.8318324	test: 0.8281938	best: 0.8281938 (14)	total: 583ms	remaining: 1m 56s
0.8307692307692307 0.5
15:	learn: 0.8318065	test: 0.8307692	best: 0.8307692 (15)	total: 618ms	remaining: 1m 55s
0.8318713450292399 0.5
16:	learn: 0.8389676	test: 0.8318713	best: 0.8318713 (16)	total: 652ms	remaining: 1m 54s
0.834061135371179 0.5
1

93:	learn: 0.8758304	test: 0.8791357	best: 0.8792987 (92)	total: 3.62s	remaining: 1m 51s
94:	learn: 0.8757880	test: 0.8791357	best: 0.8792987 (92)	total: 3.65s	remaining: 1m 51s
0.8797297297297297 0.5
95:	learn: 0.8750639	test: 0.8797297	best: 0.8797297 (95)	total: 3.7s	remaining: 1m 51s
96:	learn: 0.8754048	test: 0.8797297	best: 0.8797297 (95)	total: 3.74s	remaining: 1m 51s
0.8798920377867746 0.5
97:	learn: 0.8757880	test: 0.8798920	best: 0.8798920 (97)	total: 3.78s	remaining: 1m 51s
98:	learn: 0.8758304	test: 0.8798920	best: 0.8798920 (97)	total: 3.82s	remaining: 1m 51s
0.880647336480108 0.5
99:	learn: 0.8754473	test: 0.8806473	best: 0.8806473 (99)	total: 3.85s	remaining: 1m 51s
100:	learn: 0.8755965	test: 0.8806473	best: 0.8806473 (99)	total: 3.89s	remaining: 1m 51s
101:	learn: 0.8754897	test: 0.8806473	best: 0.8806473 (99)	total: 3.93s	remaining: 1m 51s
102:	learn: 0.8758304	test: 0.8806473	best: 0.8806473 (99)	total: 3.97s	remaining: 1m 51s
0.8808080808080808 0.5
103:	learn: 0.875

185:	learn: 0.8867128	test: 0.8814016	best: 0.8844086 (146)	total: 7.12s	remaining: 1m 47s
186:	learn: 0.8867128	test: 0.8814016	best: 0.8844086 (146)	total: 7.16s	remaining: 1m 47s
187:	learn: 0.8872384	test: 0.8814016	best: 0.8844086 (146)	total: 7.2s	remaining: 1m 47s
188:	learn: 0.8872003	test: 0.8798920	best: 0.8844086 (146)	total: 7.23s	remaining: 1m 47s
189:	learn: 0.8872003	test: 0.8798920	best: 0.8844086 (146)	total: 7.27s	remaining: 1m 47s
190:	learn: 0.8877637	test: 0.8798920	best: 0.8844086 (146)	total: 7.32s	remaining: 1m 47s
191:	learn: 0.8877637	test: 0.8798920	best: 0.8844086 (146)	total: 7.35s	remaining: 1m 47s
192:	learn: 0.8875760	test: 0.8798920	best: 0.8844086 (146)	total: 7.39s	remaining: 1m 47s
193:	learn: 0.8878757	test: 0.8798920	best: 0.8844086 (146)	total: 7.43s	remaining: 1m 47s
194:	learn: 0.8882512	test: 0.8798920	best: 0.8844086 (146)	total: 7.47s	remaining: 1m 47s
195:	learn: 0.8881757	test: 0.8783784	best: 0.8844086 (146)	total: 7.51s	remaining: 1m 47s


...

117:	learn: 0.8760669	test: 0.8739946	best: 0.8750835 (81)	total: 4.73s	remaining: 1m 55s
118:	learn: 0.8760020	test: 0.8739946	best: 0.8750835 (81)	total: 4.77s	remaining: 1m 55s
119:	learn: 0.8760020	test: 0.8739946	best: 0.8750835 (81)	total: 4.82s	remaining: 1m 55s
120:	learn: 0.8763010	test: 0.8739946	best: 0.8750835 (81)	total: 4.86s	remaining: 1m 55s
121:	learn: 0.8760443	test: 0.8732394	best: 0.8750835 (81)	total: 4.9s	remaining: 1m 55s
122:	learn: 0.8761937	test: 0.8732394	best: 0.8750835 (81)	total: 4.94s	remaining: 1m 55s
123:	learn: 0.8758103	test: 0.8732394	best: 0.8750835 (81)	total: 4.99s	remaining: 1m 55s
124:	learn: 0.8761937	test: 0.8732394	best: 0.8750835 (81)	total: 5.03s	remaining: 1m 55s
125:	learn: 0.8761937	test: 0.8726542	best: 0.8750835 (81)	total: 5.08s	remaining: 1m 55s
126:	learn: 0.8764275	test: 0.8732394	best: 0.8750835 (81)	total: 5.12s	remaining: 1m 55s
127:	learn: 0.8767263	test: 0.8726542	best: 0.8750835 (81)	total: 5.16s	remaining: 1m 55s
128:	learn:

75:	learn: 0.8713580	test: 0.8795181	best: 0.8830645 (49)	total: 2.98s	remaining: 1m 54s
76:	learn: 0.8714627	test: 0.8795181	best: 0.8830645 (49)	total: 3.02s	remaining: 1m 54s
77:	learn: 0.8718910	test: 0.8787676	best: 0.8830645 (49)	total: 3.05s	remaining: 1m 54s
78:	learn: 0.8720831	test: 0.8787676	best: 0.8830645 (49)	total: 3.09s	remaining: 1m 54s
79:	learn: 0.8722317	test: 0.8783422	best: 0.8830645 (49)	total: 3.13s	remaining: 1m 54s
80:	learn: 0.8728078	test: 0.8790915	best: 0.8830645 (49)	total: 3.17s	remaining: 1m 54s
81:	learn: 0.8726592	test: 0.8796791	best: 0.8830645 (49)	total: 3.2s	remaining: 1m 53s
82:	learn: 0.8727644	test: 0.8787676	best: 0.8830645 (49)	total: 3.24s	remaining: 1m 53s
83:	learn: 0.8724238	test: 0.8787676	best: 0.8830645 (49)	total: 3.28s	remaining: 1m 53s
84:	learn: 0.8725724	test: 0.8789298	best: 0.8830645 (49)	total: 3.32s	remaining: 1m 53s
85:	learn: 0.8725724	test: 0.8802676	best: 0.8830645 (49)	total: 3.36s	remaining: 1m 53s
86:	learn: 0.8731915	t

0.8347826086956521 0.5
20:	learn: 0.8422939	test: 0.8347826	best: 0.8347826 (20)	total: 780ms	remaining: 1m 50s
21:	learn: 0.8418231	test: 0.8327299	best: 0.8347826 (20)	total: 814ms	remaining: 1m 50s
0.8386167146974064 0.5
22:	learn: 0.8437389	test: 0.8386167	best: 0.8386167 (22)	total: 850ms	remaining: 1m 50s
0.8444444444444444 0.5
23:	learn: 0.8460039	test: 0.8444444	best: 0.8444444 (23)	total: 891ms	remaining: 1m 50s
0.8447293447293447 0.5
24:	learn: 0.8469965	test: 0.8447293	best: 0.8447293 (24)	total: 926ms	remaining: 1m 50s
0.8474095102909864 0.5
25:	learn: 0.8489437	test: 0.8474095	best: 0.8474095 (25)	total: 960ms	remaining: 1m 49s
0.849469964664311 0.5
26:	learn: 0.8493584	test: 0.8494700	best: 0.8494700 (26)	total: 997ms	remaining: 1m 49s
0.849682427664079 0.5
27:	learn: 0.8504739	test: 0.8496824	best: 0.8496824 (27)	total: 1.03s	remaining: 1m 49s
28:	learn: 0.8505868	test: 0.8496824	best: 0.8496824 (27)	total: 1.08s	remaining: 1m 50s
0.8502824858757061 0.5
29:	learn: 0.8514

107:	learn: 0.8768423	test: 0.8670757	best: 0.8670757 (100)	total: 4.45s	remaining: 1m 59s
108:	learn: 0.8768423	test: 0.8670757	best: 0.8670757 (100)	total: 4.49s	remaining: 1m 59s
109:	learn: 0.8768423	test: 0.8670757	best: 0.8670757 (100)	total: 4.53s	remaining: 1m 59s
0.8678474114441417 0.5
110:	learn: 0.8772227	test: 0.8678474	best: 0.8678474 (110)	total: 4.58s	remaining: 1m 59s
111:	learn: 0.8777100	test: 0.8663029	best: 0.8678474 (110)	total: 4.62s	remaining: 1m 59s
112:	learn: 0.8773058	test: 0.8670757	best: 0.8678474 (110)	total: 4.66s	remaining: 1m 59s
113:	learn: 0.8776028	test: 0.8670757	best: 0.8678474 (110)	total: 4.7s	remaining: 1m 58s
114:	learn: 0.8772643	test: 0.8663029	best: 0.8678474 (110)	total: 4.74s	remaining: 1m 58s
115:	learn: 0.8774128	test: 0.8663029	best: 0.8678474 (110)	total: 4.77s	remaining: 1m 58s
116:	learn: 0.8772881	test: 0.8670757	best: 0.8678474 (110)	total: 4.81s	remaining: 1m 58s
117:	learn: 0.8771395	test: 0.8670757	best: 0.8678474 (110)	total: 4

196:	learn: 0.8895270	test: 0.8677966	best: 0.8705085 (147)	total: 7.96s	remaining: 1m 53s
197:	learn: 0.8892271	test: 0.8677966	best: 0.8705085 (147)	total: 7.99s	remaining: 1m 53s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8705084746
bestIteration = 147

Shrink model to first 148 iterations.
0.5
0.9086963779001835


In [61]:
total_result2_YJ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6257 entries, 0 to 6256
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       6257 non-null   int64
 1   0       6257 non-null   int64
 2   0       6257 non-null   int64
 3   0       6257 non-null   int64
 4   0       6257 non-null   int64
 5   0       6257 non-null   int64
 6   0       6257 non-null   int64
 7   0       6257 non-null   int64
 8   0       6257 non-null   int64
 9   0       6257 non-null   int64
dtypes: int64(10)
memory usage: 489.0 KB


In [62]:
print(np.array(train_f1).mean(), np.array(val_f1).mean(), np.array(val_f1).std())

0.9070386029943514 0.8815932227619854 0.010703350242224552


## final_dataset

In [72]:
real_final_train = pd.concat([total_result1,total_result1_YJ,y_tra],axis=1)
real_final_val = pd.concat([total_result2,total_result2_YJ,y_val.reset_index(drop=True)],axis=1)
real_final_test = pd.concat([total_result3,total_result3_YJ],axis=1)

In [73]:
real_final_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53042 entries, 0 to 53041
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   0             53042 non-null  int64
 1   0             53042 non-null  int64
 2   0             53042 non-null  int64
 3   0             53042 non-null  int64
 4   0             53042 non-null  int64
 5   0             53042 non-null  int64
 6   0             53042 non-null  int64
 7   0             53042 non-null  int64
 8   0             53042 non-null  int64
 9   0             53042 non-null  int64
 10  0             53042 non-null  int64
 11  0             53042 non-null  int64
 12  0             53042 non-null  int64
 13  0             53042 non-null  int64
 14  0             53042 non-null  int64
 15  0             53042 non-null  int64
 16  0             53042 non-null  int64
 17  0             53042 non-null  int64
 18  0             53042 non-null  int64
 19  0             53042 non-n

In [74]:
real_final_val.iloc[:,-1].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6257 entries, 0 to 6256
Series name: is_converted
Non-Null Count  Dtype
--------------  -----
6257 non-null   bool 
dtypes: bool(1)
memory usage: 6.2 KB


## final catboost

In [75]:
real_final_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6257 entries, 0 to 6256
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   0             6257 non-null   int64
 1   0             6257 non-null   int64
 2   0             6257 non-null   int64
 3   0             6257 non-null   int64
 4   0             6257 non-null   int64
 5   0             6257 non-null   int64
 6   0             6257 non-null   int64
 7   0             6257 non-null   int64
 8   0             6257 non-null   int64
 9   0             6257 non-null   int64
 10  0             6257 non-null   int64
 11  0             6257 non-null   int64
 12  0             6257 non-null   int64
 13  0             6257 non-null   int64
 14  0             6257 non-null   int64
 15  0             6257 non-null   int64
 16  0             6257 non-null   int64
 17  0             6257 non-null   int64
 18  0             6257 non-null   int64
 19  0             6257 non-null

In [122]:
# total_result1_YJ = pd.DataFrame([])
# total_result2_YJ = pd.DataFrame([])
# total_result3_YJ = pd.DataFrame([])
val_f1 = []
train_f1 = []
list_t = []
n=0
num_parts=1
for i in range(num_parts):
    print(i)
#     exec(f"train_{i} = pd.concat([df_label_0_split_dfs[{i}],df_label_1]).reset_index(drop=True)")
#     exec(f"train_{i} = train_{i}.sample(len(train_{i}), random_state=42).reset_index(drop=True)")
#     X = eval(f"train_{i}").drop(columns=['is_converted'])
#     y = eval(f"train_{i}")['is_converted'].astype(int)
    
        
    X_train, X_test = real_final_train.iloc[:,:-1], real_final_val.iloc[:,:-1]
    y_train, y_test = real_final_train.iloc[:,-1], real_final_val.iloc[:,-1]

    class CatBoostEvalMetricPearson(object): 
        global global_t
        def __init__(self): 
            global global_t
            self.best_t = 0
            self.best_f1 = 0
            global_t = self.best_t

        def get_final_error(self, error, weight):
            return error

        def is_max_optimal(self):
            return True

        def evaluate(self, approxes, target, weight):
            global global_t
            assert len(approxes) == 1
            assert len(target) == len(approxes[0])
            logit_train = 1/(1+np.exp(-approxes[0]))
            target_train = np.array(target)

            best_f1 = 0
            best_t = 0
            for temp_idx in range(50,51,1): # (20, 91, 5) (50,51,1)
                temp_t = temp_idx/100
                preds_train = np.zeros(approxes[0].shape)
                preds_train[logit_train >= temp_t] = 1
                preds_train[logit_train < temp_t] = 0
                temp_f1 = f1_score(target_train, preds_train)
                if (best_f1 < temp_f1):
                    best_f1 = temp_f1
                    best_t = temp_t
                    if (self.best_f1 < best_f1) and (len(target)==len(X_test)):
                        self.best_f1 = best_f1
                        self.best_t = best_t
                        global_t = best_t
                        print(self.best_f1, self.best_t)
            return best_f1, 0


    X_train = np.array(X_train).tolist()
    X_test = np.array(X_test).tolist()
    y_train = np.array(y_train).tolist()
    y_test = np.array(y_test).tolist()
    real_final_test = np.array(real_final_test).tolist()

    model_cat = CatBoostClassifier(learning_rate = 0.001, 
                                depth = 3,
                                iterations = 3000, 
                                random_state = 45, 
                                objective='Logloss',
                                eval_metric=CatBoostEvalMetricPearson(),
                                scale_pos_weight=3,
                                one_hot_max_size=1)

    model_cat.fit(X_train, y_train, 
                eval_set=[(X_test, y_test)],
                early_stopping_rounds=100,
                use_best_model=True)

    print(global_t)
    list_t.append(global_t)

    model_cat.set_probability_threshold(global_t)
    fin_result= model_cat.predict(real_final_test)

    fin_result_np = np.array(fin_result).reshape(-1,1)

    val_f1.append(f1_score(pd.DataFrame(y_test),model_cat.predict(pd.DataFrame(X_test))))
    train_f1.append(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))
    print(f1_score(pd.DataFrame(y_train),model_cat.predict(pd.DataFrame(X_train))))

    exec(f"model_cat.save_model('model_cat_{n}')")
    n+=1
#         k_fold_on = 0
    

0
0.5700773860705073 0.5
0:	learn: 0.8638763	test: 0.5700774	best: 0.5700774 (0)	total: 39.5ms	remaining: 1m 58s
0.5729303547963206 0.5
1:	learn: 0.8706758	test: 0.5729304	best: 0.5729304 (1)	total: 76.8ms	remaining: 1m 55s
0.5820627802690582 0.5
2:	learn: 0.8684407	test: 0.5820628	best: 0.5820628 (2)	total: 114ms	remaining: 1m 54s
0.5838228648892906 0.5
3:	learn: 0.8701315	test: 0.5838229	best: 0.5838229 (3)	total: 151ms	remaining: 1m 53s
0.5849909584086799 0.5
4:	learn: 0.8729781	test: 0.5849910	best: 0.5849910 (4)	total: 189ms	remaining: 1m 52s
0.5855651384475714 0.5
5:	learn: 0.8717498	test: 0.5855651	best: 0.5855651 (5)	total: 226ms	remaining: 1m 52s
0.5904230590423059 0.5
6:	learn: 0.8722710	test: 0.5904231	best: 0.5904231 (6)	total: 263ms	remaining: 1m 52s
7:	learn: 0.8724714	test: 0.5874649	best: 0.5904231 (6)	total: 300ms	remaining: 1m 52s
8:	learn: 0.8729560	test: 0.5890603	best: 0.5904231 (6)	total: 337ms	remaining: 1m 52s
9:	learn: 0.8752530	test: 0.5888420	best: 0.5904231 

96:	learn: 0.8754123	test: 0.5809129	best: 0.5904231 (6)	total: 3.62s	remaining: 1m 48s
97:	learn: 0.8752696	test: 0.5809129	best: 0.5904231 (6)	total: 3.65s	remaining: 1m 48s
98:	learn: 0.8750318	test: 0.5809129	best: 0.5904231 (6)	total: 3.69s	remaining: 1m 48s
99:	learn: 0.8750318	test: 0.5809129	best: 0.5904231 (6)	total: 3.73s	remaining: 1m 48s
100:	learn: 0.8751746	test: 0.5809129	best: 0.5904231 (6)	total: 3.76s	remaining: 1m 48s
101:	learn: 0.8751746	test: 0.5809129	best: 0.5904231 (6)	total: 3.8s	remaining: 1m 47s
102:	learn: 0.8757457	test: 0.5809129	best: 0.5904231 (6)	total: 3.84s	remaining: 1m 47s
103:	learn: 0.8757457	test: 0.5809129	best: 0.5904231 (6)	total: 3.88s	remaining: 1m 47s
104:	learn: 0.8756029	test: 0.5809129	best: 0.5904231 (6)	total: 3.91s	remaining: 1m 47s
105:	learn: 0.8757457	test: 0.5809129	best: 0.5904231 (6)	total: 3.95s	remaining: 1m 47s
106:	learn: 0.8756029	test: 0.5809129	best: 0.5904231 (6)	total: 3.99s	remaining: 1m 47s
Stopped by overfitting det

In [123]:
fin_result

array([False, False, False, ..., False, False, False])

In [124]:
test_df = pd.read_csv("submission.csv")
test_df['is_converted'] = fin_result_np
test_df.to_csv("submission.csv", index=False)
test_df['is_converted'].value_counts()

is_converted
False    19637
True      1704
Name: count, dtype: int64